In [1]:
import numpy as np
import os
import cv2
import pandas as pd
import torch
import torch.nn as nn
import keras
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from collections import Counter
from tensorflow.python.client import device_lib
from keras.optimizers import Adam, Adamax, SGD
import imutils
import tensorflow_addons as tfa
from keras.models import load_model

2022-12-29 17:22:14.435847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-29 17:22:14.583358: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-29 17:22:14.612440: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-29 17:22:15.097870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
gpu = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)
print(gpu)
tf.keras.backend.clear_session()
TF_ENABLE_ONEDNN_OPTS=0
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(tf.device('/gpu:0'))

tf.device('/gpu:0')

In [ ]:
base_path="/home/neel/Acoustic/Acoustics/data"
spectrogram_path=os.path.sep.join([base_path,"images"])
annots_path=os.path.sep.join([base_path,"train.csv"])

base_output="/home/neel/Acoustic/Acoustics/output_mit"
model_path=os.path.sep.join([base_output,"detector.h5"])
plot_path=os.path.sep.join([base_output,"plot.png"])
test_file=os.path.sep.join([base_output,"test.txt"])

In [ ]:
# image1 = cv2.imread("/home/neel/Acoustic/yolov5_training/yolov5/runs/detect/frames_test/3m_train0_1.jpg")
# (h, w) = image1.shape[:2]
# print(h,w)

In [ ]:
print("[INFO] loading dataset...")
rows = open(annots_path).read().strip().split("\n")
spectrogram=np.empty(((len(rows)),216,216,6), dtype="float32")
bounding_box_cords=[]
filenames=[]
cnt=0
for row in rows:
    row = row.split(",")
    (filename1,filename2,X, Y, W, H) = row
    imagePath1 = os.path.sep.join([spectrogram_path, filename1])
    imagePath2 = os.path.sep.join([spectrogram_path, filename2])
    # image1 = cv2.imread(imagePath1)
    # image2=cv2.imread(imagePath2)
    # (h, w) = image1.shape[:2]

    # startX = float(startX) / w
    # startY = float(startY) / h
    # endX = float(endX) / w
    # endY = float(endY) / h

    image1 = tf.io.read_file(imagePath1)
    image1=tf.image.decode_image(image1,channels=3,dtype=tf.float32) 
    image1=tf.image.resize(image1,[216,216])
    image2 = tf.io.read_file(imagePath2)
    image2=tf.image.decode_image(image2,channels=3,dtype=tf.float32) 
    image2=tf.image.resize(image2,[216,216])
    # print(image1.shape)
    data=np.concatenate((image1,image2),axis=-1)
    spectrogram[cnt]=data / 255.0
    cnt+=1
    bounding_box_cords.append((X, Y, W, H))
    filenames.append([filename1,filename2])
    # filenames.append(filename1)
print("Done.")

In [ ]:
# data1 = np.array(spectrogram1, dtype="float32") / 255.0    #(example , height, width, rgb*channel) = (316,640,640,3*2)
# data2 = np.array(spectrogram2, dtype="float32") / 255.0 
# targets = np.array(bounding_box_cords, dtype="float32")
# # data1.shape   #(example, cords) = (316,4)
# data=np.concatenate((data1,data2),axis=-1)
# data.shape
targets = np.array(bounding_box_cords, dtype="float32")
print(targets.shape)
del bounding_box_cords

In [ ]:
split = train_test_split(spectrogram, targets, filenames, test_size=0.10,random_state=42)
del spectrogram, targets,filenames
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]

print("[INFO] saving testing filenames...")
f = open(test_file, "w")
for i in testFilenames:
    f.write(i[0]+","+i[1])
    f.write("\n")
# f.write("\n".join(testFilenames))
f.close()

In [ ]:
print(trainTargets)

In [2]:
model= tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32,(3,3), padding='valid',strides=2, activation="relu", input_shape=(216,216,6)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='valid'))

model.add(tf.keras.layers.Conv2D(32,(3,3), padding='valid',strides=2, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='valid', strides=2, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), padding='valid'))

model.add(tf.keras.layers.Conv2D(64,(3,3), padding='same', activation="relu"))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(128,(3,3), padding='same',strides=2, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2D(256,(3,3), padding='same', strides=2, activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2D(512,(3,3), padding='valid', strides=2, activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2D(512,(3,3), padding='valid',strides=2, activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2D(1024,(3,3), padding='valid', activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2D(1024,(3,3), padding='same', activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

# model.add(tf.keras.layers.Conv2DTranspose(1024,(3,3), padding='valid', activation="relu"))
# model.add(tf.keras.layers.Conv2DTranspose(1024,(3,3), padding='valid', activation="relu"))
# model.add(tf.keras.layers.Conv2DTranspose(1024,(3,3), padding='valid',strides=2, activation="relu"))
# model.add(tf.keras.layers.Conv2DTranspose(1024,(3,3), padding='valid', activation="relu"))

# model.add(tf.keras.layers.Conv2D(512,(3,3), padding='same', activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(256,(3,3), padding='same', activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.Conv2D(125,(3,3), padding='same', activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dense(4, activation="relu"))

2022-12-29 17:22:17.949715: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-29 17:22:19.688301: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 210 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:19:00.0, compute capability: 8.6


In [3]:
init_lr=1e-4

epoch=60
batch_size=32
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    init_lr,
    decay_steps=20,
    decay_rate=0.1,
    staircase=True)

opt = SGD(learning_rate=lr_schedule ,momentum=0.9, )
model.compile(
  optimizer='sgd',
  loss='mse',
  metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 107, 107, 32)      1760      
                                                                 
 batch_normalization (BatchN  (None, 107, 107, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 53, 53, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 32)       128       
 hNormalization)                                                 
                                                        

In [ ]:
with tf.device("/gpu:0"):
	H = model.fit(
		trainImages, trainTargets,
		validation_data=(testImages, testTargets),
		batch_size=batch_size,
		epochs=epoch,
		verbose=1)
	print(H)

In [ ]:
import matplotlib.pyplot as plt
print("[INFO] saving object detector model...")
model.save(model_path, save_format="h5")
N = epoch
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_loss")
plt.title("Bounding Box Regression MSE on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot_path)

In [ ]:
# model=load_model(model_path,custom_objects={'custom_mse': custom_mse})
model=load_model(model_path)
for img in trainFilenames:
    imagePath1=os.path.sep.join([spectrogram_path, img[0]])
    imagePath2=os.path.sep.join([spectrogram_path, img[1]])
    image1 = tf.io.read_file(imagePath1)
    image1=tf.image.decode_image(image1,channels=3,dtype=tf.float32) 
    image1=tf.image.resize(image1,[216,216])
    image2 = tf.io.read_file(imagePath2)
    image2=tf.image.decode_image(image2,channels=3,dtype=tf.float32) 
    image2=tf.image.resize(image2,[216,216])
    image1 = np.expand_dims(image1, axis=0)
    image2 = np.expand_dims(image2, axis=0)
    image=np.concatenate((image1,image2),axis=-1)

    preds = model.predict(image)[0]
    (startX, startY, endX, endY) = preds
    # image2 = cv2.imread(os.path.sep.join(["/home/neel/Acoustic/yolov5_training/img_data/frames", img[0][9:]]))
    # image2 = imutils.resize(image2, width=600)
    # (h, w) = image1.shape[:2]
    # startX = startX*w
    # startY = startY*h
    # endX = endX*w
    # endY = endY*h
    f=open("/home/neel/Acoustic/Acoustics/data/labels/"+img[0][9:-4]+".txt")
    ground_truth=f.read()
    print("Ground truth: ",ground_truth[1:])
    print("Predicted: ",startX, startY, endX, endY)
    # cv2.rectangle(image2, (startX, startY), (endX, endY),(0, 255, 0), 2)
	# # show the output image
    # cv2.imwrite("/home/neel/Acoustic/Acoustics/output/test_img/"+img[0], image2)
    # cv2.imshow("Output", image2)
    # cv2.waitKey(0)

In [ ]:
def bbox_iou(box1, box2, xywh=True, GIoU=False, DIoU=False, CIoU=False, eps=1e-7):
    # Returns Intersection over Union (IoU) of box1(1,4) to box2(n,4)

    # Get the coordinates of bounding boxes
    if xywh:  # transform from xywh to xyxy
        (x1, y1, w1, h1), (x2, y2, w2, h2) = box1.chunk(4, 1), box2.chunk(4, 1)
        w1_, h1_, w2_, h2_ = w1 / 2, h1 / 2, w2 / 2, h2 / 2
        b1_x1, b1_x2, b1_y1, b1_y2 = x1 - w1_, x1 + w1_, y1 - h1_, y1 + h1_
        b2_x1, b2_x2, b2_y1, b2_y2 = x2 - w2_, x2 + w2_, y2 - h2_, y2 + h2_
    else:  # x1, y1, x2, y2 = box1
        b1_x1, b1_y1, b1_x2, b1_y2 = box1.chunk(4, 1)
        b2_x1, b2_y1, b2_x2, b2_y2 = box2.chunk(4, 1)
        w1, h1 = b1_x2 - b1_x1, b1_y2 - b1_y1
        w2, h2 = b2_x2 - b2_x1, b2_y2 - b2_y1

    # Intersection area
    inter = (torch.min(b1_x2, b2_x2) - torch.max(b1_x1, b2_x1)).clamp(0) * \
            (torch.min(b1_y2, b2_y2) - torch.max(b1_y1, b2_y1)).clamp(0)

    # Union Area
    union = w1 * h1 + w2 * h2 - inter + eps

    # IoU
    iou = inter / union
    if CIoU or DIoU or GIoU:
        cw = torch.max(b1_x2, b2_x2) - torch.min(b1_x1, b2_x1)  # convex (smallest enclosing box) width
        ch = torch.max(b1_y2, b2_y2) - torch.min(b1_y1, b2_y1)  # convex height
        if CIoU or DIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            c2 = cw ** 2 + ch ** 2 + eps  # convex diagonal squared
            rho2 = ((b2_x1 + b2_x2 - b1_x1 - b1_x2) ** 2 + (b2_y1 + b2_y2 - b1_y1 - b1_y2) ** 2) / 4  # center dist ** 2
            if CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi ** 2) * torch.pow(torch.atan(w2 / (h2 + eps)) - torch.atan(w1 / (h1 + eps)), 2)
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                return iou - (rho2 / c2 + v * alpha)  # CIoU
            return iou - rho2 / c2  # DIoU
        c_area = cw * ch + eps  # convex area
        return iou - (c_area - union) / c_area  # GIoU https://arxiv.org/pdf/1902.09630.pdf
    return iou  # IoU